# Convert dihedral coeff in lopls

from opls style to multi/harmonic style.

unit is not changed here

## Conversion between RB-style dihedral function and Fourier (opls) style

In [3]:


def opls2RB(Flist):
    # convert the LAMMPS's "opls" dihedral form to RB form which is used in Gromacs
    # v_opls = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2 + f4*(1-cos(4fi))/2
    f0 = 0
    f1 = Flist[0]
    f2 = Flist[1]
    f3 = Flist[2]
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3]    


# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def Fourier2RB(Flist):
#     # lammps standard opls style dihedral:
#     # v_dihedral = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
# #     Flist = [f1,f2,f3,f4]
#     [f1,f2,f3,f4] = Flist
#     c0 = f2 + (f1+f3)/2.0
#     c1 = 0.5 * (-1.0*f1+3*f3)
#     c2 = -1.0*f2 + 4*f4
#     c3 = -2 * f3

#     return [c0,c1,c2,c3]



def Fourier2RB_Siu(Flist):
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    #
    [f0,f1,f2,f3] = Flist
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3]


# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def RB2Fourier(RBlist):
#     # convert RB dihedral form to lammps's opls standard form
#     # RB: v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
#     [c0,c1,c2,c3] = RBlist

#     f3 = -0.5*c3
#     f2 = -c2
#     f1 = -1.5*c3-2*c1

#     return [f1,f2,f3,0]


def RB2Fourier_Siu(RBlist):
    # convert RB dihedral form to Siu 2012's opls form
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    [c0,c1,c2,c3] = RBlist
    f3 = -0.5*c3
    f2 = -c2
    f1 = -1.5*c3-2*c1
    f0 = c0 - 0.5*(f1+2*f2+f3)
    
    return [f0,f1,f2,f3]

## Unit conversion

In [17]:
def kcal2kj (value):
    """
    convert unit from kcal/mol to kJ/mol 
    1 kcal = 4.184 kJ
    """

    return (float(value)*4.184)


def kj2kcal (value):
    """
    convert unit from kj/mol to kcal/mol 
    1 kcal = 4.184 kJ
    """
    
    return (float(value)/4.184)


def ev2kj (value):
    """
    convert unit from eV to kJ/mol 
    1 eV = 96 kJ/mol
    """
    
    return float(value)*96

In [23]:
opls2RB([0,0,0.198])

[0.099, 0.29700000000000004, 0, -0.396]

In [20]:
with open("old_dihedral_coeff.txt") as file:
    data = file.readlines()
    
for line in data:
    word = line.split()
    Flist = [float(word[2]),float(word[3]),float(word[4])]
#     print(Flist)
    c = opls2RB(Flist)
    print(f"{word[0]} {word[1]} multi/harmonic {c[0]:7.5f} {c[1]:7.5f} {c[2]:7.5f} {c[3]:7.5f}")

dihedral_coeff @dihedral:013L_013L_013L_013L multi/harmonic 0.19711 -0.05502 0.21434 -0.35644
dihedral_coeff @dihedral:X_047L_047L_X multi/harmonic 12.25026 -0.00000 -12.25026 -0.00000
dihedral_coeff @dihedral:047L_047L_013L_013L multi/harmonic -0.13225 0.24759 -0.32189 0.20655
dihedral_coeff @dihedral:013L_013L_013L_047L multi/harmonic 0.46434 0.02552 -0.13437 -0.35549
dihedral_coeff @dihedral:046L_013LL_005L_007L multi/harmonic 0.18080 0.56682 0.01456 -0.76218
dihedral_coeff @dihedral:046L_013LL_013LL_005L multi/harmonic 0.17364 0.39312 -0.03302 -0.53374
dihedral_coeff @dihedral:013LL_013LL_005L_007L multi/harmonic -0.16734 0.89769 0.01604 -0.74640
dihedral_coeff @dihedral:013LL_013LL_013LL_005L multi/harmonic 0.70893 0.30050 0.26631 -1.27573
dihedral_coeff @dihedral:005L_013LL_013LL_005L multi/harmonic 0.85869 -0.25693 0.84971 -1.45146
dihedral_coeff @dihedral:013LL_003L_020L_013LL multi/harmonic 7.29732 -1.55961 -5.73771 -0.00000
dihedral_coeff @dihedral:013LL_020L_003L_004L multi/

In [10]:
c

[0.7576649999999999, -0.6589049999999999, 2.37742, -2.47618]